# Plot the planned/micro-manip/ephys trajectories for each mouse

In [28]:
import unityneuro.render as urn
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
import ibllib.atlas as atlas
NeedlesAtlas = atlas.NeedlesAtlas(25)

import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

TEXT_X = -600
TEXT_Y_INC = 30

In [26]:
def getCoords(ins, coord_transform):
    entry_coords = coord_transform.xyz2ccf(ins.entry)
    tip_coords = coord_transform.xyz2ccf(ins.tip)
    angles = [ins.phi, ins.theta, ins.beta]
    # depth = vec_dist(entry_coords, tip_coords)
    return (entry_coords, tip_coords, angles)

def traj2coords(traj, coord_transform):
    insertion = atlas.Insertion.from_dict(traj)
    return getCoords(insertion, coord_transform)

def plot_session_histology(mouse):
    bwm_mm = one.alyx.rest('trajectories', 'list', provenance='Micro-manipulator', subject=mouse,
                          project='ibl_neuropixel_brainwide_01', use_cache=False)

    urn.clear()

    # get all the insertions for this mouse
    urn.set_area_visibility({'8':True})
    urn.set_area_material({'8':'transparent-unlit'})
    urn.set_area_alpha({'8':0.15})
    
    probe_names = []
    probet_texts = {}
    probet_colors = {}
    probet_positions = {}

    for i, ins in enumerate(bwm_mm):
        # get the coords
        (entry, tip, angles) = traj2coords(ins, NeedlesAtlas)
        
        pname = 'mm'+str(i)
        urn.create_probes([pname])
        urn.set_probe_positions({pname:list(tip)})
        urn.set_probe_angles({pname:list(angles)})
        color = randHexColor()
        urn.set_probe_colors({pname:color})
        probe_names.append(pname)
        probet_texts[pname] = ins['id'][0:8]
        probet_colors[pname] = color
        probet_positions[pname] = [TEXT_X,i*TEXT_Y_INC]
        
    urn.create_text(probe_names)
    urn.set_text(probet_texts)
    urn.set_text_colors(probet_colors)
    urn.set_text_positions(probet_positions)
        
    urn.create_text(['mouse_name'])
    urn.set_text({'mouse_name':mouse})
    urn.set_text_colors({'mouse_name':'#FFA500'})

    urn.set_camera_rotation(45,0,0)
    
    return bwm_mm

In [ ]:
urn.setup()

In [27]:
bwm_mm = plot_session_histology('NR_0020')